#           Finding Replacement Players for Oakland A’s
__Faraz Khan__,

__May 18, 2020__


## Introduction

In this project I will be providing a solution to the actual problem a baseball team Oakland A’s faced in 2001 when three of their key players left the team. To find the replacement players,the general manager of Oakland A’s took sports analytics to the next level by inventing some new KPIs for finding undervalued but brilliant players . This strategy was so successful that they went on the win 20 consecutive games. This strategy was brought to the world in a book called ‘Moneyball: The Art of Winning an Unfair Game’ by Michel Lewis. Later, this book was turned into a movie called Moneyball.
In this project we will analyze the KPIs that better account for the offence’s success such as Slugging and On Base Percentage. While finding replacement players we have following three constraints:

1. Combined salaries should not exceed 15 million USD
2. Combined At Bat(AB) should be more than the combined AB of lost players
3. Mean On Base Percentage(OBP) should be more than the mean OBP of lost players

__PlayerID of lost players : giambja01 , damonjo01 , saenzol01__

Before we start, Let's import some packages that might help us in this task

In [12]:
import pandas as pd
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import matplotlib.ticker as ticker
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import jaccard_similarity_score
from sklearn import metrics

## Loading the Datasets and analyzing it's structure.

In [13]:
batting = pd.read_csv("Batting.csv")
salaries = pd.read_csv("Salaries.csv")
batting.head()

,playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,...,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
0,aardsda01,2004,1,SFN,NL,11,11.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0
1,aardsda01,2006,1,CHN,NL,45,43.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,45.0
2,aardsda01,2007,1,CHA,AL,25,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,aardsda01,2008,1,BOS,AL,47,5.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0
4,aardsda01,2009,1,SEA,AL,73,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [14]:
salaries.head()

,yearID,teamID,lgID,playerID,salary
0,1985,BAL,AL,murraed02,1472819
1,1985,BAL,AL,lynnfr01,1090000
2,1985,BAL,AL,ripkeca01,800000
3,1985,BAL,AL,lacyle01,725000
4,1985,BAL,AL,flanami01,641667


Let's assess the distribution of variables in these 2 datasets.

In [15]:
print('Summary of Batting')
print(batting.describe())
print('Summary of Salaries')
print(salaries.describe())

Summary of Batting
             yearID         stint             G     G_batting            AB  \
count  97889.000000  97889.000000  97889.000000  96483.000000  91476.000000   
mean    1961.732922      1.076873     51.654078     49.130790    154.067766   
std       38.104588      0.282653     47.267487     48.869353    187.374936   
min     1871.000000      1.000000      1.000000      0.000000      0.000000   
25%     1931.000000      1.000000     13.000000      7.000000      9.000000   
50%     1970.000000      1.000000     35.000000     32.000000     61.000000   
75%     1995.000000      1.000000     81.000000     81.000000    260.000000   
max     2013.000000      5.000000    165.000000    165.000000    716.000000   

                  R             H            2B            3B            HR  \
count  91476.000000  91476.000000  91476.000000  91476.000000  91476.000000   
mean      20.468615     40.366883      6.799543      1.424483      3.001640   
std       28.935336     53.67448

## Adding new statistical measures to the dataset:

In [16]:
#Adding a new variable OBP for calculating On Base Percentage:
batting['OBP'] = (batting['H']+batting['BB']+batting['HBP'])/(batting['AB']+batting['BB']+batting['HBP']+batting['SF'])

#Adding a new variable SLG for calculating Slugging:
#Creating a new variable 1xB to calculate Singles as it is not already in the batting table.
batting['1B']=batting['H']-(batting['2B']+batting['3B']+batting['HR'])

#Creating new column for slugging using following formula
batting['SLG']=(batting['1B']+(2*batting['2B'])+(3*batting['3B'])+(4*batting['HR']))/batting['AB']

#Quick look at the final batting table:
batting.head()

,playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,...,SO,IBB,HBP,SH,SF,GIDP,G_old,OBP,1B,SLG
0,aardsda01,2004,1,SFN,NL,11,11.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,NaN,0.0,NaN
1,aardsda01,2006,1,CHN,NL,45,43.0,2.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,45.0,0.0,0.0,0.0
2,aardsda01,2007,1,CHA,AL,25,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,0.0,NaN
3,aardsda01,2008,1,BOS,AL,47,5.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
4,aardsda01,2009,1,SEA,AL,73,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN


## Cleaning and merging datasets:

Since our Salary data starts from year 1985, we will only select data after 1984 from batting table

In [17]:
batting=batting[batting["yearID"] > 1984]

#Checking if we applied subset function correctly and only have data after 1984
batting["yearID"].describe()

count    35652.000000
mean      1999.724391
std          8.178206
min       1985.000000
25%       1993.000000
50%       2000.000000
75%       2007.000000
max       2013.000000
Name: yearID, dtype: float64

In [18]:
#Merging batting and salary data for further analysis on selecting replacement players
batsal=batting.merge(salaries,how='outer',left_on=['playerID','yearID'],right_on=['playerID','yearID'])

#Removing NaN values.
batsal.dropna(inplace=True)

#Checking if everything is in order.
batsal.head(25)

,playerID,yearID,stint,teamID_x,lgID_x,G,G_batting,AB,R,H,...,SH,SF,GIDP,G_old,OBP,1B,SLG,teamID_y,lgID_y,salary
3,aardsda01,2008,1.0,BOS,AL,47.0,5.0,1.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.000000,0.0,0.000000,BOS,AL,403250.0
11,aasedo01,1989,1.0,NYN,NL,49.0,49.0,5.0,0.0,0.0,...,0.0,0.0,0.0,49.0,0.000000,0.0,0.000000,NYN,NL,400000.0
15,abadan01,2006,1.0,CIN,NL,5.0,5.0,3.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.400000,0.0,0.000000,CIN,NL,327000.0
20,abbotje01,1998,1.0,CHA,AL,89.0,89.0,244.0,33.0,68.0,...,2.0,5.0,2.0,89.0,0.298450,41.0,0.491803,CHA,AL,175000.0
21,abbotje01,1999,1.0,CHA,AL,17.0,17.0,57.0,5.0,9.0,...,1.0,1.0,4.0,17.0,0.222222,7.0,0.263158,CHA,AL,255000.0
22,abbotje01,2000,1.0,CHA,AL,80.0,80.0,215.0,31.0,59.0,...,2.0,1.0,2.0,80.0,0.343096,40.0,0.395349,CHA,AL,255000.0
23,abbotje01,2001,1.0,FLO,NL,28.0,28.0,42.0,5.0,11.0,...,0.0,0.0,1.0,28.0,0.326087,8.0,0.333333,FLO,NL,300000.0
34,abbotji01,1999,1.0,MIL,NL,20.0,18.0,21.0,0.0,2.0,...,3.0,0.0,1.0,18.0,0.095238,2.0,0.095238,MIL,NL,400000.0
35,abbotku01,1993,1.0,OAK,AL,20.0,20.0,61.0,11.0,15.0,...,3.0,0.0,3.0,20.0,0.281250,11.0,0.409836,OAK,AL,109000.0
36,abbotku01,1994,1.0,FLO,NL,101.0,101.0,345.0,41.0,86.0,...,3.0,2.0,5.0,101.0,0.290761,57.0,0.394203,FLO,NL,109000.0


## Having a look at lost players statistics:

Let's make a subset of lost players

In [19]:
#Subselecting data of players lost in 2001
lost_players=batting[(batting['playerID']=='giambja01')| \
                                                    (batting['playerID']=='damonjo01') | (batting['playerID']=='saenzol01')]
lost_players=lost_players[lost_players['yearID']==2001]
lost_players

,playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,...,SO,IBB,HBP,SH,SF,GIDP,G_old,OBP,1B,SLG
19745,damonjo01,2001,1,OAK,AL,155,155.0,644.0,108.0,165.0,...,70.0,1.0,5.0,5.0,4.0,7.0,155.0,0.323529,118.0,0.363354
30725,giambja01,2001,1,OAK,AL,154,154.0,520.0,109.0,178.0,...,83.0,24.0,13.0,0.0,9.0,17.0,154.0,0.476900,91.0,0.659615
76245,saenzol01,2001,1,OAK,AL,106,106.0,305.0,33.0,67.0,...,64.0,1.0,13.0,1.0,3.0,9.0,106.0,0.291176,36.0,0.383607


Let's calculate some statistics for lost players:

In [20]:
print('OBP mean of lost players:',lost_players['OBP'].mean())
print('Combined AB of lost players:',lost_players['AB'].sum())

OBP mean of lost players: 0.36386867712807924
Combined AB of lost players: 1469.0


## Excluding lost players from the dataset and selecting replacements:

In [25]:
#Excluding lost players from the analysis
delete_row_1 = batsal[batsal["playerID"]=='giambja01'].index
delete_row_2 = batsal[batsal["playerID"]=='damonjo01'].index
delete_row_3 = batsal[batsal["playerID"]=='saenzol01'].index
remaining_players = batsal.drop(delete_row_1)
remaining_players = remaining_players.drop(delete_row_2)
remaining_players = remaining_players.drop(delete_row_3)

#Selecting replacement players according to the defined criteria
replacement_players=remaining_players[remaining_players['AB']>300]
replacement_players=replacement_players[replacement_players['yearID']==2001]
replacement_players=replacement_players[replacement_players['OBP']>0.37]
replacement_players=replacement_players[replacement_players['salary']<=5000000]

#Arraning for highest OBP
replacement_players.sort_values(by='OBP',ascending=False,inplace=True)

#displaying top 3 choices
Top_3=replacement_players.head(3)
Top_3

,playerID,yearID,stint,teamID_x,lgID_x,G,G_batting,AB,R,H,...,SH,SF,GIDP,G_old,OBP,1B,SLG,teamID_y,lgID_y,salary
13287,heltoto01,2001,1.0,COL,NL,159.0,159.0,587.0,132.0,197.0,...,1.0,5.0,14.0,159.0,0.431655,92.0,0.684838,COL,NL,4950000.0
2446,berkmla01,2001,1.0,HOU,NL,156.0,156.0,577.0,110.0,191.0,...,0.0,6.0,8.0,156.0,0.430233,97.0,0.620451,HOU,NL,305000.0
11336,gonzalu01,2001,1.0,ARI,NL,162.0,162.0,609.0,128.0,198.0,...,0.0,5.0,14.0,162.0,0.428571,98.0,0.688013,ARI,NL,4833333.0


Let's confirm our choices by checking if they meet our constraints.

In [30]:
print('OBP mean of Replacement:',Top_3['OBP'].mean())
print('Combined AB of Replacement:',Top_3['AB'].sum())
print('Combined Salaries of Replacement:',Top_3['salary'].sum())

OBP mean of Replacement: 0.43015288765665205
Combined AB of Replacement: 1773.0
Combined Salaries of Replacement: 10088333.0


## Conclusion:

In the end we have successfully chosen three palyers who meet our original criteria we set in the introduction. These replacement players are some of the great players who are highly undervalued in the market. We have unearthed them thanks to the innovative features.

Further details about the players is provided in the links with their names below.

__1.Todd Helton__

https://www.baseball-reference.com/players/h/heltoto01.shtml

__2.Lance Berkman__

https://www.baseball-reference.com/players/b/berkmla01.shtml

__3.Luis Gonzalez__

https://www.baseball-reference.com/players/g/gonzalu01.shtml